## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 284s 10ms/step - loss: 0.2259 - accuracy: 0.9211 - val_loss: 0.0562 - val_accuracy: 0.9819
Epoch 2/5
29404/29404 [==============================] - 134s 5ms/step - loss: 0.0686 - accuracy: 0.9782 - val_loss: 0.0444 - val_accuracy: 0.9846
Epoch 3/5
29404/29404 [==============================] - 328s 11ms/step - loss: 0.0484 - accuracy: 0.9852 - val_loss: 0.0279 - val_accuracy: 0.9903
Epoch 4/5
29404/29404 [==============================] - 288s 10ms/step - loss: 0.0392 - accuracy: 0.9883 - val_loss: 0.0224 - val_accuracy: 0.9924
Epoch 5/5
29404/29404 [==============================] - 478s 16ms/step - loss: 0.0333 - accuracy: 0.9895 - val_loss: 0.0236 - val_accuracy: 0.9918
Training time: 0:25:15.070847
Test score: 0.02360828135437908
Test accuracy: 0.9917712211608887


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the 
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

/home/amazon/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 177s 6ms/step - loss: 0.1534 - accuracy: 0.9568 - val_loss: 0.0182 - val_accuracy: 0.9940
Epoch 2/5
30596/30596 [==============================] - 159s 5ms/step - loss: 0.0429 - accuracy: 0.9879 - val_loss: 0.0122 - val_accuracy: 0.9957
Epoch 3/5
30596/30596 [==============================] - 156s 5ms/step - loss: 0.0300 - accuracy: 0.9910 - val_loss: 0.0077 - val_accuracy: 0.9967
Epoch 4/5
30596/30596 [==============================] - 154s 5ms/step - loss: 0.0244 - accuracy: 0.9926 - val_loss: 0.0071 - val_accuracy: 0.9977
Epoch 5/5
30596/30596 [==============================] - 153s 5ms/step - loss: 0.0215 - accuracy: 0.9937 - val_loss: 0.0056 - val_accuracy: 0.9982
Training time: 0:13:25.968155
Test score: 0.005586164730068739
Test accuracy: 0.9982486963272095


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
### To do:
- Now write code to reverse this training process.  That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [16]:
model_2 = Sequential(feature_layers + classification_layers)

In [17]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [18]:
train_model(model_2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 165s 5ms/step - loss: 0.0166 - accuracy: 0.9950 - val_loss: 0.0051 - val_accuracy: 0.9981
Epoch 2/5
30596/30596 [==============================] - 170s 6ms/step - loss: 0.0169 - accuracy: 0.9948 - val_loss: 0.0044 - val_accuracy: 0.9986
Epoch 3/5
30596/30596 [==============================] - 167s 5ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.0043 - val_accuracy: 0.9986
Epoch 4/5
30596/30596 [==============================] - 164s 5ms/step - loss: 0.0129 - accuracy: 0.9966 - val_loss: 0.0040 - val_accuracy: 0.9986
Epoch 5/5
30596/30596 [==============================] - 170s 6ms/step - loss: 0.0115 - accuracy: 0.9961 - val_loss: 0.0037 - val_accuracy: 0.9990
Training time: 0:14:01.604241
Test score: 0.0037076063272686696
Test accuracy: 0.9990270733833313


In [19]:
# Freeze only the 
for l in feature_layers:
    l.trainable = False

In [20]:
train_model(model_2,
            (x_train_gte5,y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 161s 5ms/step - loss: 0.1195 - accuracy: 0.9665 - val_loss: 0.0261 - val_accuracy: 0.9914
Epoch 2/5
29404/29404 [==============================] - 139s 5ms/step - loss: 0.0407 - accuracy: 0.9877 - val_loss: 0.0230 - val_accuracy: 0.9922
Epoch 3/5
29404/29404 [==============================] - 134s 5ms/step - loss: 0.0320 - accuracy: 0.9900 - val_loss: 0.0214 - val_accuracy: 0.9942
Epoch 4/5
29404/29404 [==============================] - 135s 5ms/step - loss: 0.0262 - accuracy: 0.9913 - val_loss: 0.0204 - val_accuracy: 0.9944
Epoch 5/5
29404/29404 [==============================] - 139s 5ms/step - loss: 0.0253 - accuracy: 0.9918 - val_loss: 0.0184 - val_accuracy: 0.9951
Training time: 0:11:53.150612
Test score: 0.018397683480340862
Test accuracy: 0.9950627684593201
